In [1]:
cd ../src

/gpfs/home/gdb20/code/batch-manager/src


In [2]:
import input_files
import input_combi
import helpers
import os

In [5]:
#a library of templates would be nice

batch_runner_configs = {
    'max_jobs' : 10,
    'verbosity' : False,
    'job_basename' : 'mike'
}

global_configs = { '' : {
        'num_cores' : 8,
        'mem_per_cpu_GB' : 4,
        'runtime' : '2-00:00:00',
        #'!max_jobs' : 10, #this doesn't do anything yet
        #'solvent' : 'thf', #this is what Mike told me we use
    }
}

thf = { 'thf' : {
    'solvent' : 'thf',
    }
}

xyz_root = '/gpfs/home/gdb20/code/mol-maker/data/mike/mike_molecules'
mike_dir = os.path.join(xyz_root,'')

mike_mols = input_combi.xyz_files_from_directory(mike_dir)
mike_mols['!directories'] = True



orbital_and_uno_occs_blocks = {
    'output' : [
        'print[P_UNO_OccNum] 1'
    ],    
    'plots'  : [
        'dim1 100',
        'dim2 100',
        'dim3 100',
        'Format Gaussian_Cube',
        'SpinDens("spindensity");',
        'ElDens("electrondensity");',
    ]
}   

tddft_block = {
    'tddft' : [
        'maxdim 5',
        'nroots 50',
        'DoNTO true',
        'NTOStates 1,2,3,4,5,6,7,8,9,10', #somewhat arbitrary choice
    ],
}

diffuse_linear_dependency_blocks = {
    'scf' : [
        'sthresh 1e-6',
    ],
    'basis' : [
        'PCDTrimBas overlap',
        'PCDTrimAuxJ coulomb',
        'PCDTrimAuxJK coulomb',
        #'PCDTrimAuxC coulomb', #for use with correlated calculations
        'PCDThresh -1', #-1 means automatically select threshold
    ]
}


orca_defaults = {
    'program' : 'ORCA',
    'integration_grid' : 'DEFGRID2', #default settings
    'scf_tolerance' : 'TightSCF', #see if they work in ORCA 6
}

orca_singlepoint = orca_defaults.copy()
orca_singlepoint.update({      
    'natural_orbitals' : True,
    'runtime' : '0-05:00:00',
    'blocks' : orbital_and_uno_occs_blocks,   
    'run_type' : None,
})

orca_opt_freq = orca_defaults.copy()
orca_opt_freq.update({
    'runtime' : '3-00:00:00',
    'run_type' : 'OPT FREQ',
})

orca_tddft = orca_defaults.copy()
orca_tddft.update({ #more than 5 days is way too much
    'runtime' : '1-12:00:00',
    'run_type' : None,
    'blocks' : tddft_block,
})


orca_nics = orca_defaults.copy()
orca_nics.update({
    'runtime' : '5-00:00:00', #some of these will fail, dang. took like, AN HOUR last time we did this, so weird
    'run_type' : 'NMR', #GIAO is the default for ORCA
})

gaussian_nics = {
    'program'  : 'Gaussian',
    'runtime'  : '3-00:00:00',
    'run_type' : 'NMR=GIAO', 
}

gaussian_nics_preprocessing = {
    'num_cores' : 1,
    'mem_per_cpu_GB' : 3,
    'runtime' : '0-00:05:00',
    'program'  : 'pyAroma',
    'cc_program' : 'Gaussian',
}

orca_nics_preprocessing = {
    'num_cores' : 1,
    'mem_per_cpu_GB' : 3,
    'runtime' : '0-00:05:00',
    'program'  : 'pyAroma',
    'cc_program' : 'ORCA',
}


r2scan_3c = {
    'functional' : 'r2SCAN-3c',    
    'basis' : None,
    'aux_basis' : None,
}

cam_b3lyp = {
    'functional' : 'CAM-B3LYP',
    'basis' : '6-31G(d,p)',
    'aux_basis' : 'autoaux',
}

wb97x_3c = {
    'functional' : 'wB97X-3c',
    'basis_set' : None,
    'aux_basis' : None,
}
neutral_singlet = {
    'charge' : 0,
    'spin_multiplicity' : 1,
    'broken_symmetry' : False,
}

neutral_triplet = {
    'charge' : 0,
    'spin_multiplicity' : 3,
    'broken_symmetry' : False,
}

neutral_singlet_brokensym = {
    'charge' : 0,
    'spin_multiplicity' : 3,
    'broken_symmetry' : True,
}

neutral_singlet_guessmix = {
    'charge' : 0,
    'spin_multiplicty' : 1,
    'uks' : True,
    'keywords' : ['guess=mix']
}

crest_quick_settings = {
    'program' : 'CREST',
    'functional' : 'gfn2',
    'quick' : 'quick',
    'reopt' : True,
    'cluster' : True,
    'noreftopo' : True, #might regret this
}

xtb_cm_states = {
    '0_1' : neutral_singlet,
    '0_3': neutral_triplet,
}
orca_cm_states = xtb_cm_states.copy()
orca_cm_states.update({
    '0_1' : neutral_singlet_brokensym,
})

gaussian_cm_states = xtb_cm_states.copy()
gaussian_cm_states.update({
    '0_1' : neutral_singlet_guessmix,
})


In [6]:
crest_gfn2__gfnff_quick = crest_quick_settings.copy()
crest_gfn2__gfnff_quick.update({        
    'functional' : 'gfn2//gfnff',
})

crude_confsearch_optfreq = {
    'crest_gfn2__gfnff_quick' : crest_gfn2__gfnff_quick,
    'xtb_gfn2_opt_freq' : {
        'program' : 'XTB',
        'functional' : 'gfn2',
        'run_type' : 'ohess', #opt-freq
        '!coords_from' : '../crest_gfn2__gfnff_quick',
        '!xyz_file' : 'crest_best.xyz', #these are sort of bad...
        #'!overwrite' : True,
    },
}

#SINGLEPOINTS
r2scan_3c_triplet_sp = orca_singlepoint.copy()
r2scan_3c_triplet_sp.update({          
        **r2scan_3c,
        **neutral_triplet,   
        '!coords_from' : '../xtb_gfn2_opt_freq',
        '!xyz_file' : 'xtbopt.xyz',
        #'!overwrite' : True, #VERY DANGEROUS, comment out when not using     
})

r2scan_3c_singlet_sp = orca_singlepoint.copy()
r2scan_3c_singlet_sp.update({ 
        **r2scan_3c,
        **neutral_singlet_brokensym,
        '!coords_from' : '../xtb_gfn2_opt_freq',
        '!xyz_file' : 'xtbopt.xyz',
        #'!overwrite' : True,  #VERY DANGEROUS, comment out when done using  
})

r2scan_3c_tddft = orca_tddft.copy() #let's only do one of these
r2scan_3c_tddft.update({
    **r2scan_3c,
    'runtime' : '0-05:00:00',
    '!coords_from' : '../xtb_gfn2_opt_freq',
    '!xyz_file' : 'xtbopt.xyz',
})

orca_nics_prepro = orca_nics_preprocessing.copy()
orca_nics_prepro.update({
    '!coords_from' : '../xtb_gfn2_opt_freq',
    '!xyz_file' : 'xtbopt.xyz',
    #'!overwrite' : True,  DO NOT
})

r2scan_3c_nics = orca_nics.copy()
r2scan_3c_nics.update({
    **r2scan_3c,
    'runtime' : '0-05:00:00',
    '!coords_from' : '../orca_prepro',
    '!xyz_file'    : 'orca_prepro.xyz',
    #'!overwrite' : 'all', 
})

orca_workflow_crude = { #here we assign filenames to each group of settings
    'r2scan_3c_triplet_sp_orbitals' : r2scan_3c_triplet_sp,
    'r2scan_3c_bs_singlet_sp_orbitals_occs' : r2scan_3c_singlet_sp,
    'r2scan_3c_tddft' : r2scan_3c_tddft,
    'nics_prepro' : orca_nics_prepro,
    'r2scan_3c_nics' : r2scan_3c_nics,
}
minimal_crude = {
    'r2scan_3c_triplet_sp_orbitals' : r2scan_3c_triplet_sp,
    'r2scan_3c_bs_singlet_sp_orbitals_occs' : r2scan_3c_singlet_sp,
}


In [7]:
croot_dir ='/gpfs/research/alabuginlab/gage/michael/crude/'

input_combi.do_everything(
    croot_dir,
    batch_runner_configs,
    [global_configs,xtb_cm_states,mike_mols,crude_confsearch_optfreq],
    [global_configs,orca_cm_states,mike_mols,orca_workflow_crude],
)

IF I HAD ONE
crest_best.xyz
IF I HAD ONE
crest_best.xyz
IF I HAD ONE
crest_best.xyz
IF I HAD ONE
crest_best.xyz
IF I HAD ONE
crest_best.xyz
IF I HAD ONE
crest_best.xyz
IF I HAD ONE
crest_best.xyz
IF I HAD ONE
crest_best.xyz
IF I HAD ONE
crest_best.xyz
IF I HAD ONE
crest_best.xyz
IF I HAD ONE
crest_best.xyz
IF I HAD ONE
crest_best.xyz
IF I HAD ONE
crest_best.xyz
IF I HAD ONE
crest_best.xyz
IF I HAD ONE
crest_best.xyz
IF I HAD ONE
crest_best.xyz
IF I HAD ONE
crest_best.xyz
IF I HAD ONE
crest_best.xyz
IF I HAD ONE
crest_best.xyz
IF I HAD ONE
crest_best.xyz
IF I HAD ONE
crest_best.xyz
IF I HAD ONE
crest_best.xyz
IF I HAD ONE
crest_best.xyz
IF I HAD ONE
crest_best.xyz
IF I HAD ONE
crest_best.xyz
IF I HAD ONE
crest_best.xyz
IF I HAD ONE
crest_best.xyz
IF I HAD ONE
crest_best.xyz
IF I HAD ONE
crest_best.xyz
IF I HAD ONE
crest_best.xyz
IF I HAD ONE
crest_best.xyz
IF I HAD ONE
crest_best.xyz
IF I HAD ONE
crest_best.xyz
IF I HAD ONE
crest_best.xyz
IF I HAD ONE
crest_best.xyz
IF I HAD ONE
crest_b

In [8]:
cam_b3lyp_6_31gdp_opt_freq = orca_opt_freq.copy()
cam_b3lyp_6_31gdp_opt_freq.update({
        **cam_b3lyp,
        'run_type' : 'COPT FREQ', #cartesian optimization
        '!coords_from' : '../crest_gfn2_quick',
        '!xyz_file' : 'crest_best.xyz',
        'runtime' : '4-00:00:00',
        #'!overwrite' : 'not_succeeded',
})

cam_b3lyp_6_31gdp_triplet_sp = orca_singlepoint.copy()
cam_b3lyp_6_31gdp_triplet_sp.update({
        **neutral_triplet,
        **cam_b3lyp,
        '!coords_from' : '../cam-b3lyp_6-31Gdp_opt_freq',
        '!xyz_file' : 'cam-b3lyp_6-31Gdp_opt_freq.xyz',
        #'!overwrite' : True, #this is very hardcore, comment out unless using
})

cam_b3lyp_6_31gdp_singlet_sp = orca_singlepoint.copy()
cam_b3lyp_6_31gdp_singlet_sp.update({
        **neutral_singlet_brokensym,
        **cam_b3lyp,
        '!coords_from' : '../cam-b3lyp_6-31Gdp_opt_freq',
        '!xyz_file' : 'cam-b3lyp_6-31Gdp_opt_freq.xyz',
        #'!overwrite' : True, #comment out until using, avoid accidentally removing files
})

cam_b3lyp_diffuse = cam_b3lyp.copy()
cam_b3lyp_diffuse.update({'basis' : '6-31+G(d,p)'})
cam_b3lyp_6_31_plus_gdp_tddft = orca_tddft.copy()
cam_b3lyp_6_31_plus_gdp_tddft.update({
    **cam_b3lyp_diffuse,
    'blocks' : diffuse_linear_dependency_blocks, #these all also failed for the same reason
    'scf_tolerance' : 'VeryTightSCF',
    '!coords_from' : '../cam-b3lyp_6-31Gdp_opt_freq',
    '!xyz_file' : 'cam-b3lyp_6-31Gdp_opt_freq.xyz',
    #'!overwrite' : 'not_succeeded', #leaving this uncommented is like leaving your keys outside
})


########JOBS FOR DOING NICS
# gauss_nics_prepro = gaussian_nics_preprocessing.copy()
# gauss_nics_prepro.update({
#     'xyz_file' :  'cam-b3lyp_6-31Gdp_opt_freq.xyz',
#     '!coords_from' : '../cam-b3lyp_6-31Gdp_opt_freq',
#     '!xyz_file' : 'cam-b3lyp_6-31Gdp_opt_freq.xyz',
#     #'!overwrite' : True, #these are fine, but we'll need to give some of them more time 
# })
# cam_b3lyp_6_31_plus_gdp_NICS_gaussian = gaussian_nics.copy()
# cam_b3lyp_6_31_plus_gdp_NICS_gaussian.update({
#     **cam_b3lyp_diffuse,
#     '!coords_from' : '../gaussian_prepro',
#     '!xyz_file'    : 'gaussian_prepro.xyz', #I don't like the way this works. will change
#     #'!overwrite' : True,  #DO NOT OVERWRITE
# })
# cam_b3lyp_6_31_plus_gdp_NICS_gaussian.update({
#     'aux_basis' : None,
# })

orca_nics_prepro = orca_nics_preprocessing.copy()
orca_nics_prepro.update({
    'xyz_file' :  'cam-b3lyp_6-31Gdp_opt_freq.xyz', #yes, we have to do this twice for these, once without '!' and once with
    '!coords_from' : '../cam-b3lyp_6-31Gdp_opt_freq',
    '!xyz_file' : 'cam-b3lyp_6-31Gdp_opt_freq.xyz',
    #'!overwrite' : True,  DO NOT
})



cam_b3lyp_6_31_plus_gdp_NICS = orca_nics.copy()
cam_b3lyp_6_31_plus_gdp_NICS.update({
    **cam_b3lyp_diffuse,
    'blocks' : diffuse_linear_dependency_blocks,
    'scf_tolerance' : 'VeryTightSCF',
    '!coords_from' : '../orca_prepro',
    '!xyz_file'    : 'orca_prepro.xyz',
    #'!overwrite' : 'all', 
})

#SETTINGS FOR ITERATING IN INPUT_COMBI
crest_gfn2_quick = {
    'crest_gfn2_quick' : crest_quick_settings,
}

orca_workflow_b3lyp = {
    'cam-b3lyp_6-31Gdp_opt_freq' : cam_b3lyp_6_31gdp_opt_freq, 
    'cam-b3lyp_6-31Gdp_triplet_sp_orbitals' : cam_b3lyp_6_31gdp_triplet_sp,
    'cam-b3lyp_6-31Gdp_bs_sp_orbitals_occs' : cam_b3lyp_6_31gdp_singlet_sp,
    'cam-b3lyp_6-31+Gdp_tddft' : cam_b3lyp_6_31_plus_gdp_tddft,
    'orca_prepro' : orca_nics_prepro,
    'cam-b3lyp_6-31+Gdp_NICS' : cam_b3lyp_6_31_plus_gdp_NICS,
}
# gaussian_cam_b3lyp_nics = {
#     'gaussian_prepro' : gauss_nics_prepro,
#     'cam-b3lyp_6-31+Gdp_NICS_gaussian' : cam_b3lyp_6_31_plus_gdp_NICS_gaussian,
# }


In [9]:
b3lyp_dir = '/gpfs/research/alabuginlab/gage/michael/quick/'

#b3lyp
input_combi.do_everything(
    b3lyp_dir,
    batch_runner_configs,
    [global_configs,xtb_cm_states,mike_mols,crest_gfn2_quick],
    [global_configs,orca_cm_states,mike_mols,orca_workflow_b3lyp],
    # [global_configs,gaussian_cm_states,mike_mols,gaussian_cam_b3lyp_nics],
)

IF I HAD ONE
IF I HAD ONE
IF I HAD ONE
IF I HAD ONE
IF I HAD ONE
IF I HAD ONE
IF I HAD ONE
IF I HAD ONE
IF I HAD ONE
IF I HAD ONE
IF I HAD ONE
IF I HAD ONE
IF I HAD ONE
IF I HAD ONE
IF I HAD ONE
IF I HAD ONE
IF I HAD ONE
IF I HAD ONE
IF I HAD ONE
IF I HAD ONE
IF I HAD ONE
IF I HAD ONE
IF I HAD ONE
IF I HAD ONE
IF I HAD ONE
IF I HAD ONE
IF I HAD ONE
IF I HAD ONE
IF I HAD ONE
IF I HAD ONE
IF I HAD ONE
IF I HAD ONE
IF I HAD ONE
IF I HAD ONE
IF I HAD ONE
IF I HAD ONE
editing batchfile
crest_best.xyz
cam-b3lyp_6-31Gdp_opt_freq.xyz
cam-b3lyp_6-31Gdp_opt_freq.xyz
cam-b3lyp_6-31Gdp_opt_freq.xyz
cam-b3lyp_6-31Gdp_opt_freq.xyz
orca_prepro.xyz
crest_best.xyz
cam-b3lyp_6-31Gdp_opt_freq.xyz
cam-b3lyp_6-31Gdp_opt_freq.xyz
cam-b3lyp_6-31Gdp_opt_freq.xyz
cam-b3lyp_6-31Gdp_opt_freq.xyz
orca_prepro.xyz
crest_best.xyz
cam-b3lyp_6-31Gdp_opt_freq.xyz
cam-b3lyp_6-31Gdp_opt_freq.xyz
cam-b3lyp_6-31Gdp_opt_freq.xyz
cam-b3lyp_6-31Gdp_opt_freq.xyz
orca_prepro.xyz
crest_best.xyz
cam-b3lyp_6-31Gdp_opt_freq.xyz
cam

In [10]:
crest = {
    'crest' : {
        'program' : 'CREST',
        'functional' : 'gfn2',
        'quick' : False,
        'cluster' : True,
        'noreftopo' : True,
    }
}

r2SCAN_3c_opt_freq = orca_opt_freq.copy()
r2SCAN_3c_opt_freq.update({
        **r2scan_3c,
        'run_type' : 'OPT FREQ',
        '!coords_from' : '../crest',
        '!xyz_file' : 'crest_best.xyz',
        #'!overwrite' : 'not_succeeded',
})

wB97X_3c_sp_singlet = orca_singlepoint.copy()
wB97X_3c_sp_singlet.update({ #this one will include options for rendering orbitals,
    **neutral_singlet_brokensym,
    **wb97x_3c,
    '!coords_from' : '../r2SCAN-3c_opt_freq',
    '!xyz_file' : 'r2SCAN-3c_opt_freq.xyz',
    #'!overwrite' : 'all', #hazardous to sanity to leave these uncommented when not using!
}),

#this had ...
wB97X_3c_sp_triplet = orca_singlepoint.copy()
wB97X_3c_sp_triplet.update({
    **neutral_triplet,
    **wb97x_3c,
    '!coords_from' : '../r2SCAN-3c_opt_freq',
    '!xyz_file' : 'r2SCAN-3c_opt_freq.xyz',
    #'!overwrite' : 'all', #COMMENT THEM OUT UNTIL USING
})

wB97X_3c_tddft = orca_tddft.copy()
wB97X_3c_tddft.update({
    **wb97x_3c,
    '!coords_from' : '../r2SCAN-3c_opt_freq',
    '!xyz_file' : 'r2SCAN-3c_opt_freq.xyz',
    #'!overwrite' : True,
})


#NICS

orca_nics_prepro = orca_nics_preprocessing.copy()
orca_nics_prepro.update({
    'xyz_file' :  'r2SCAN-3c_opt_freq.xyz', #yes, we have to do this twice for these, once without '!' and once with
    '!coords_from' : '../r2SCAN-3c_opt_freq',
    '!xyz_file' : 'r2SCAN-3c_opt_freq.xyz',
    #'!overwrite' : True,  #DO NOT
})

wB97X_3c_nics = orca_nics.copy()
wB97X_3c_nics.update({
    **wb97x_3c,
    '!coords_from' : '../orca_nics_prepro',
    '!xyz_file'    : 'orca_nics_prepro.xyz',
    #'!overwrite' : True,
})

orca_compound = {
    'r2SCAN-3c_opt_freq' : r2SCAN_3c_opt_freq,
    'wB97X-3c_sp_singlet' : wB97X_3c_sp_singlet,
    'wB97X-3c_sp_triplet' : wB97X_3c_sp_triplet,
    'wB97X-3c_tddft' : wB97X_3c_tddft,
    'orca_nics_prepro' : orca_nics_prepro,
    'wB97X-3c_nics' : wB97X_3c_nics,
}

In [11]:
orca_compound_dir = '/gpfs/research/alabuginlab/gage/michael/compound/'
#compound
new_br_configs = batch_runner_configs.copy()
new_br_configs.update({'max_jobs' : 5})
input_combi.do_everything(
    orca_compound_dir,
    new_br_configs,
    [global_configs,thf,xtb_cm_states,mike_mols,crest],
    [global_configs,thf,orca_cm_states,mike_mols,orca_compound],
)

IF I HAD ONE
IF I HAD ONE
IF I HAD ONE
IF I HAD ONE
IF I HAD ONE
IF I HAD ONE
IF I HAD ONE
IF I HAD ONE
IF I HAD ONE
IF I HAD ONE
IF I HAD ONE
IF I HAD ONE
IF I HAD ONE
IF I HAD ONE
IF I HAD ONE
IF I HAD ONE
IF I HAD ONE
IF I HAD ONE
IF I HAD ONE
IF I HAD ONE
IF I HAD ONE
IF I HAD ONE
IF I HAD ONE
IF I HAD ONE
IF I HAD ONE
IF I HAD ONE
IF I HAD ONE
IF I HAD ONE
IF I HAD ONE
IF I HAD ONE
IF I HAD ONE
IF I HAD ONE
IF I HAD ONE
IF I HAD ONE
IF I HAD ONE
IF I HAD ONE
editing batchfile
crest_best.xyz
r2SCAN-3c_opt_freq.xyz
r2SCAN-3c_opt_freq.xyz
r2SCAN-3c_opt_freq.xyz
r2SCAN-3c_opt_freq.xyz
orca_nics_prepro.xyz
crest_best.xyz
r2SCAN-3c_opt_freq.xyz
r2SCAN-3c_opt_freq.xyz
r2SCAN-3c_opt_freq.xyz
r2SCAN-3c_opt_freq.xyz
orca_nics_prepro.xyz
crest_best.xyz
r2SCAN-3c_opt_freq.xyz
r2SCAN-3c_opt_freq.xyz
r2SCAN-3c_opt_freq.xyz
r2SCAN-3c_opt_freq.xyz
orca_nics_prepro.xyz
crest_best.xyz
r2SCAN-3c_opt_freq.xyz
r2SCAN-3c_opt_freq.xyz
r2SCAN-3c_opt_freq.xyz
r2SCAN-3c_opt_freq.xyz
orca_nics_prepro.xyz
cr

In [12]:
test_dir = '/gpfs/research/alabuginlab/gage/michael/test/'
test_molecules = {
    '0_1_omy' : {
        'xyz_file' : 'omy.xyz',
        'xyz_directory' : '/gpfs/home/gdb20/code/mol-maker/data/mike/mike_molecules',
    },
    '0_3_cby' : {
        'xyz_file' : 'cmy.xyz',
        'xyz_directory' : '/gpfs/home/gdb20/code/mol-maker/data/mike/mike_molecules',
    },
    '!directories' : True
}

cam_b3lyp_6_31gdp_opt_freq.update({
 # '!overwrite' : True,  
})
orca_cam_b3lyp_opt_freq_sp = {
    'cam-b3lyp_6-31Gdp_opt_freq' : cam_b3lyp_6_31gdp_opt_freq, 
}
input_combi.do_everything( #these all only requested 4 cores, that's a problem
    [global_configs,test_molecules,orca_cam_b3lyp_opt_freq_sp],
    test_dir,
)
cam_b3lyp_6_31gdp_opt_freq.update({
  #'!overwrite' : False,
})

creating script for run


TypeError: 'str' object is not a mapping

In [13]:

hbpo_croot_dir ='/gpfs/research/alabuginlab/gage/michael/nanographenes/crude/'

hpbo_molecules_dir = '/gpfs/home/gdb20/code/mol-maker/data/mike/nanographenes'
hpbo_mols = input_combi.xyz_files_from_directory(hpbo_molecules_dir)
hpbo_mols['!directories'] = True


thf.update({'!directories':False})
xtb_cm_states.update({'!directories' : False})
orca_cm_states.update({'!directories': False})

hbpo_b3lyp_dir = '/gpfs/research/alabuginlab/gage/michael/nanographenes/b3lyp/'


batch_runner_configs.update({'max_jobs' : 4, 'job_basename' : 'ng_b3lyp'})
input_combi.do_everything(
    hbpo_b3lyp_dir,
    batch_runner_configs,
    [global_configs,xtb_cm_states,hpbo_mols,crest_gfn2_quick],
    [global_configs,orca_cm_states,hpbo_mols,orca_workflow_b3lyp],
    # [global_configs,gaussian_cm_states,mike_mols,gaussian_cam_b3lyp_nics],
)


solvent_list = { 
    'thf' : {'solvent' : 'thf'},
    'chcl3' : {'solvent' : 'chcl3'},
    'toluene' : {'solvent' : 'toluene'},
    'ch2cl2' : {'solvent' : 'ch2cl2'},
    '!directories' : True
}

thf_list = { 
    'thf' : {'solvent' : 'thf'},
    '!directories' : True
}

xtb_cm_states.update({'!directories' : True})
orca_cm_states.update({'!directories': True})
batch_runner_configs.update({'max_jobs' : 4, 'job_basename' : 'ng_crude'})
input_combi.do_everything(
    hbpo_croot_dir,
    batch_runner_configs,
    [global_configs,hpbo_mols,solvent_list,xtb_cm_states,crude_confsearch_optfreq],
    [global_configs,hpbo_mols,solvent_list,orca_cm_states,minimal_crude],
    [global_configs,hpbo_mols,thf_list,orca_cm_states,orca_workflow_crude],
)


hbpo_orca_compound_dir = '/gpfs/research/alabuginlab/gage/michael/nanographenes/compound/'
#compound
new_br_configs = batch_runner_configs.copy()
new_br_configs.update({'max_jobs' : 4, 'job_basename' : 'ng_compound'})
input_combi.do_everything(
    hbpo_orca_compound_dir,
    new_br_configs,
    [global_configs,thf,hpbo_mols,xtb_cm_states,crest],
    [global_configs,thf,hpbo_mols,orca_cm_states,orca_compound],
)

IF I HAD ONE
IF I HAD ONE
IF I HAD ONE
IF I HAD ONE
IF I HAD ONE
IF I HAD ONE
IF I HAD ONE
IF I HAD ONE
editing batchfile
crest_best.xyz
cam-b3lyp_6-31Gdp_opt_freq.xyz
cam-b3lyp_6-31Gdp_opt_freq.xyz
cam-b3lyp_6-31Gdp_opt_freq.xyz
cam-b3lyp_6-31Gdp_opt_freq.xyz
orca_prepro.xyz
crest_best.xyz
cam-b3lyp_6-31Gdp_opt_freq.xyz
cam-b3lyp_6-31Gdp_opt_freq.xyz
cam-b3lyp_6-31Gdp_opt_freq.xyz
cam-b3lyp_6-31Gdp_opt_freq.xyz
orca_prepro.xyz
crest_best.xyz
cam-b3lyp_6-31Gdp_opt_freq.xyz
cam-b3lyp_6-31Gdp_opt_freq.xyz
cam-b3lyp_6-31Gdp_opt_freq.xyz
cam-b3lyp_6-31Gdp_opt_freq.xyz
orca_prepro.xyz
crest_best.xyz
cam-b3lyp_6-31Gdp_opt_freq.xyz
cam-b3lyp_6-31Gdp_opt_freq.xyz
cam-b3lyp_6-31Gdp_opt_freq.xyz
cam-b3lyp_6-31Gdp_opt_freq.xyz
orca_prepro.xyz
crest_best.xyz
cam-b3lyp_6-31Gdp_opt_freq.xyz
cam-b3lyp_6-31Gdp_opt_freq.xyz
cam-b3lyp_6-31Gdp_opt_freq.xyz
cam-b3lyp_6-31Gdp_opt_freq.xyz
orca_prepro.xyz
crest_best.xyz
cam-b3lyp_6-31Gdp_opt_freq.xyz
cam-b3lyp_6-31Gdp_opt_freq.xyz
cam-b3lyp_6-31Gdp_opt_freq